In [1]:
import asyncio
from lsst.ts import salobj
import logging

In [2]:
import time

In [3]:
d = salobj.Domain()

In [4]:
r = salobj.Remote(d, "Environment", 1)

In [5]:
await r.start_task

In [ ]:
settings = await r.evt_settingVersions.next(flush=False, timeout=2)

In [ ]:
print(settings)

In [10]:
await asyncio.sleep(1)
for i in range(3):
    print(await r.evt_heartbeat.next(flush=True, timeout=30))

EnvironmentID: 1, private_revCode: 0819bc35, private_sndStamp: 1576185320.788324, private_rcvStamp: 1576185320.7895663, private_seqNum: 1, private_origin: 141, private_host: -1947882788, heartbeat: False, priority: 0
EnvironmentID: 1, private_revCode: 0819bc35, private_sndStamp: 1576185321.7906392, private_rcvStamp: 1576185321.7918355, private_seqNum: 2, private_origin: 141, private_host: -1947882788, heartbeat: False, priority: 0
EnvironmentID: 1, private_revCode: 0819bc35, private_sndStamp: 1576185322.7930477, private_rcvStamp: 1576185322.7941663, private_seqNum: 3, private_origin: 141, private_host: -1947882788, heartbeat: False, priority: 0


In [11]:
await asyncio.sleep(1.)
await salobj.set_summary_state(r, salobj.State.ENABLED, settingsToApply="default", timeout=60.)

[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [ ]:
await r.cmd_setLogLevel.set_start(level=logging.DEBUG)

In [ ]:
err = await r.evt_errorCode.next(flush=False, timeout=1)

In [ ]:
err.errorReport

In [ ]:
print(err.traceback)

In [ ]:
await asyncio.sleep(1.)
await salobj.set_summary_state(r, salobj.State.STANDBY)

In [9]:
await asyncio.sleep(1.)
await salobj.set_summary_state(r, salobj.State.OFFLINE)

RemoteEvent(Environment, 1, heartbeat) falling behind; read 97 messages


[<State.ENABLED: 2>,
 <State.DISABLED: 1>,
 <State.STANDBY: 5>,
 <State.OFFLINE: 4>]

In [12]:
await asyncio.sleep(1.)
w = await r.tel_weather.next(flush=True, timeout=61)

In [ ]:
print(w)

In [ ]:
wind = await r.tel_windSpeed.next(flush=False, timeout=61)

In [ ]:
print(wind)

In [ ]:
wind = await r.tel_windDirection.next(flush=False, timeout=61)

In [ ]:
print(wind)

In [ ]:
wind = await r.tel_windGustDirection.next(flush=False, timeout=61)

In [ ]:
print(wind)

In [ ]:
%%script bash 
make_idl_files.py Environment

In [ ]:
loop = asyncio.get_event_loop()

In [ ]:
time1 = loop.time()

In [ ]:
await asyncio.sleep(1.)

In [ ]:
loop.time()

In [ ]:
time.time()

If CSC goes to FAULT state restart it and restart the remote as well. 

In [ ]:
r.evt_summaryState.flush()

In [ ]:
async def Watch_Env():
    
    print("Create domain and remote")
    d = salobj.Domain()
    r = salobj.Remote(d, "Environment", 1)
    print("Wait for start_task")
    await r.start_task
    print("Wait for heartbeat from the CSC")
    for i in range(3):
        print(await r.evt_heartbeat.next(flush=True, timeout=30))

    print("Enable CSC")
    await r.cmd_setLogLevel.set_start(level=logging.DEBUG)
    await salobj.set_summary_state(r, salobj.State.ENABLED, settingsToApply="default", timeout=60.)

    r.evt_summaryState.flush()

    min_delta = 300.

    time_0 = time.time()

    while True:
        ss = await r.evt_summaryState.next(flush=False)
        time_1 = time.time()
        if time_1 - time_0 < min_delta:
            print("CSC went to FAULT too quick. Giving up....")
            break
        elif ss.summaryState == salobj.State.FAULT:
            print("CSC in FAULT state. Bring to OFFLINE state")
            await salobj.set_summary_state(r, salobj.State.OFFLINE)
            print("Close remote and domain")
            await r.close()
            await d.close()
            print("Recreate domain and remote")
            d = salobj.Domain()
            r = salobj.Remote(d, "Environment", 1)
            print("Wait for start_task")
            await r.start_task
            print("Wait for heartbeat from the CSC")
            for i in range(3):
                print(await r.evt_heartbeat.next(flush=True, timeout=30))

            print("Reenable CSC")
            await r.cmd_setLogLevel.set_start(level=logging.DEBUG)
            await salobj.set_summary_state(r, salobj.State.ENABLED, settingsToApply="default", timeout=60.)
        else:
            print(f"CSC state: {salobj.State(ss.summaryState)}")

In [ ]:
await Watch_Env()